Improvements:

Because this is all about blending; it can probably be done completely in latent space. This might make it faster and more detailed maybe?

Use a loss to push down the alpha values - we want them small.

If we use non-random timesteps can we get larger diversity?

Make the loss more straightfoward - using the decoder. This should improve image quality even when using raster!

Do a grid of backgrounds + foregrounds (disentanble them all at once)

This can all be done during the 3d process as well - though it appears we proably don't have to??

3 way compositions:
X Y in Z where X<{jesus, kermit, obama} and Y<{driving a car, eating a sandwich, shooting a gun} in {heaven, hell, australia}

Investigate: Why does this work without the greenscreen prompt suffixes?

Allow movement/zooming via weighted averages of small changes in posisition - between steps, we don't apply the weighted average - instead we apply the average weighted translation 

Neat tricks: an image that looks good at any hue (randomize hue shift during optimization...or maybe during regular diffusion who knows)

In [ ]:
import rp

import nerf.sd as sd
import torch
import torch.nn as nn
from source.learnable_textures import LearnableTexturePackRaster,LearnableTexturePackFourier
from source.learnable_textures import LearnableImageRaster,LearnableImageFourier
import icecream
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

In [ ]:
if 's' not in dir():
    s=sd.StableDiffusion('cuda:2')
    # s=sd.StableDiffusion('cuda:0','nitrosocke/Arcane-Diffusion')
device=s.device

In [ ]:
prompts=[
    # 'arcane style, jinx wearing goggles', 
    # 'beautiful sks hatsune miku, Greg Rutkowski, ilya kuvshinov,artgerm highly detailed',
    # 'arcane style, beautiful sks hatsune miku, ilya kuvshinov,artgerm highly detailed',
    # 'arcane style, beautiful sks gay rainbow dragon, ilya kuvshinov,artgerm highly detailed',
    # 'arcane style, dragon',
    # 'arcane style, shitting his cranberries, jean luc picard of the starship enterprise, in a sea of cranberries',
    # 'arcane style, shrek shrek shrek shrek shrek shrek',
    # 'arcane style, korra from legend of korra',
    # 'arcane style, emma watson holding a puppy',
    'arcane style, sonic the hedgehog',
    # "arcane style, hatsune miku",
    # "arcane style, a puppy with two heads",
    # "arcane style, a cylon",
    # "kurokiyousei hatsune miku",
    # "hatsune miku",
    # "shrek"
]*3

try:
    for prompt in prompts:
        image = s.prompt_to_img(prompt)[0]
        with rp.SetCurrentDirectoryTemporarily(rp.make_directory('untracked/image_dump')):
            print('Saved:',
                  rp.save_image(image,
                                '%s__%i'%(prompt, rp.millis())
                               )
                 )
        rp.display_image(image)
except KeyboardInterrupt:
    print("Cancelled")
    pass
print("Done")

In [ ]:
class ColorBlendedLearnableImage(nn.Module):
    def __init__(self,image,color=(0,0,0),learnable_image=None):
        
        super().__init__()
        
        # if isinstance(image,str):
            # S=256
            # image=rp.load_image(image,use_cache=True)
            # image=rp.cv_resize_image(image,(S,S))
        
        # assert rp.is_image(image)
        assert len(color)==3
        # image=rp.as_rgb_image(rp.as_float_image(image))
        height=width=512
        # height=width=256
        # height=width=128
        r,g,b=color
        
        # self.foreground=rp.as_torch_image(image).to(device)
        # self.foreground=LearnableImageFourier(256,256).to(device)
        
        if learnable_image is None:
            self.learnable_image=LearnableImageFourier(height,width,num_channels=4,num_features=512).to(device)#RGBA
            # self.learnable_alpha=LearnableImageRaster(height,width,num_channels=1)
        else:
            self.learnable_image=learnable_image
        
        self.background=torch.zeros(3,height,width,dtype=torch.float32).to(device)
        self.background[0]=r
        self.background[1]=g
        self.background[2]=b
    
    def forward(self):
        im=self.learnable_image()
        
        
        alpha=im[3:,:,:,]
        # alpha=alpha.clamp(0,1)
        
        foreground=im[:3,:,:]
        # foreground=foreground.mean(0,keepdim=True).repeat(3,1,1)#Grayscale it!

        output=foreground*alpha + (1-alpha) * self.background
        return output
    
    


In [ ]:
#Cartridge: Custom
image_url="https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png"

X='a woman wearing a tall beige hat with purple feathers'
X='hatsune miku as a mouse'
X='a magical surreal cottage'
X='a photograph of a human skull'
# X='a cyberpunk assassin girl'
# X='an award winning makeup tutorial'
X='the tree of life'
X="a 3d kawaii cyberpunk girl's face"
X="cyberpunk lolita outfit"
X='a 3d steampunk cat head made of brass gears and bolts, trending on artstation'
X='a spectacular photo of a nuclear blast mushroom cloud, trending on artstation'#Failed once
X='A staircase to heaven, long tall twisting. Beautiful spiral staircase into the clouds, into the gates of heaven. Trending on artstation, unreal engine'
X="cupcake themed lolita outfit. a beautiful lolita dress with accessories. kawaii lolita dress. cure lolita outfit."
X="a cute kitten in a cardboard box. Trending on artstation, unreal engine"
# X='a photograph of a puppy'
X='a photograph of a teddy bear eating a baguette'
X='a photograph of a teddy bear wielding a giant flaming claymore'
X='a photograph of a teddy bear wielding a giant sword and shield'
X='a photograph of a dog in a mecha suit. a mecha doge!'
X='a photograph of a spooky skeleton doing a jojo pose'
X='a photograph of two skeletons kissing'
X='a gay rainbow dragon, 3d unreal engine trending on artstation'
X='complex metal truss scaffolding bridge, 3d unreal engine trending on artstation'
X='complex rusty metal truss scaffolding radio tower, 3d unreal engine trending on artstation'
X='anime, 3d character design, wild hot east asian woman, ghibli studio, big beautiful eyes, black skirt, tinted glasses, cool, attitude, bershka, teki, cinematic lighting, hyperreal octane render, bape, emo, black shirt, supreme, detailed, hd, 4k'
X='3d character design , rainbow hair , long hair , wild hot woman, ghibli studio, symbols tattoos on body, partly in armor, blue tinted glasses , cool, attitude, techwear , teki, bape, supreme, detailed, hd, 4k'
X='a big sailboat, hypermaximalistic, high details, cinematic, 8k resolution, beautiful detailed, insanely intricate details, artstation trending, octane render, unreal engine'
X='beautiful young russian female model, boudoir, red and black formal business suit, sensual, studio lighting, trending on instagram, full body, detailed and intricate, super detailed artstation scifi scene an old ward,one  ceiling fan, wheelchair, crutches,bed, dust, paneled walls,window,summer unreal engine 5, hyper realism, realistic shading, cinematic composition, blender render, octane render, hdr, detailed textures, photorealistic, wide shot portrait of a spanish conquistador in battle, by Daniel Zrom, masamune shirow, josan gonzales and dan mumford, ayami kojima, takato yamamoto, barclay shaw, karol bak, yukito kishiro scifi  advanced mushroom city streets, modern architecture, by Michael Parkes, concept art powerful male clown, willem dafoe as the joker, full body character concept, covered in full metal armor, art nouveau, super powers, fantasy, intricate, elegant, highly detailed, digital painting, artstation, concept art, shining, sharp focus, illustration, art oleg bulakh the world war i, surrealistic detailed claymation art, dark, moody, foggy '
X='concept art for a car with saw blades on the sides, illustrated by syd mead, high quality'
X='a low poly tree with cubes as fruits'
# X='Minecraft House Ideas on blank canvas, beautiful bright lighting, god rays, bloom effect with volumetric lighting, trending on artstation, artstation RTX, artstation graphic'
# X='planet saturn lithograph by adolphe millot'
# X='picture of Flat universe and 3D Planet popping out with galaxy and star background'
X='three identical planet earths arranged at 9 0 degrees from each other, realistic photograph'
X='Danny devito'
X='violin'
X='helicopter'
X='a flying pig wearing aviator goggles, barreling torwards the camera with a smile on its face. A pig flying torwards the camera wearing flight goggles'
X='a metallic silver dalek'
X='portrait Anime Boy in mechanical armor steampunk cute-fine-face, pretty face, realistic shaded Perfect face, fine details. Anime. Bioshock steampunk realistic shaded lighting by katsuhiro otomo ghost-in-the-shell, magali villeneuve, artgerm, rutkowski Jeremy Lipkin and Giuseppe Dangelico Pino and Michael Garmash and Rob Re'
X='Sushi'
X='Hamburger'
X='A wooden wheelbarrow full of cabbages'
X='A wooden wheelbarrow full of puppies'
X='Cotton candy ice cream cone'
X='a hamburger made of gravel and mud'
X='shrek'
X='pikachu'
X='Hollow knight'
X='mario'
X='the tree of life'
X='arcane style, jinx wearing goggles'
X='arcane style, jinx '
X='arcane style, dragon'
X='arcane style, emma watson holding a puppy'



# X='two spooky skeletons kissing'
# X='emma watson kissing a skeleton'

# X='an award winning photo of jesus eating an avocado'
# X='an avocado armchair'
# X='saturn and neptune'
# X='a close-up photo of the planet saturn'

text_black=s.get_text_embeddings('X in deep space. A X clip art, on a black background'.replace('X',X))
text_white=s.get_text_embeddings('X clip art with bright light. X on a white background'.replace('X',X))

# text_red  =s.get_text_embeddings('a photograph of a X on a solid red background'.replace('X',X))
# text_green=s.get_text_embeddings('a photograph of a X on a solid green background'.replace('X',X))
# text_blue =s.get_text_embeddings('a X on a solid blue background'.replace('X',X))

text_red  =s.get_text_embeddings('X on a red screen, stock image. royalty free picture of X on a solid red background.'.replace('X',X))
text_gree =s.get_text_embeddings('X on a green screen, stock image. royalty free picture of X on a solid green background.'.replace('X',X))
text_blue =s.get_text_embeddings('X on a blue screen, stock image. royalty free picture of X on a solid blue background.'.replace('X',X))

text_blue=text_red=text_green=text_white=text_black=s.get_text_embeddings('X'.replace('X',X))#Do the prompts even matter?



def do_display(skip=1,img_only=False):
    ds_imgs=[]
    ds_imgs.append(rp.as_numpy_image(image_black()))
    ds_imgs.append(rp.as_numpy_image(image_white()))
    ds_imgs.append(rp.as_numpy_image(image_red  ()))
    ds_imgs.append(rp.as_numpy_image(image_green()))
    ds_imgs.append(rp.as_numpy_image(image_blue ()))
    ds_imgs.append(rp.as_numpy_image(image_alpha()))
    ds_imgs.append(rp.as_numpy_image(image_full ()))
    ds_imgs=rp.labeled_images(ds_imgs,['On black','On white','On red','On green','On blue','Alpha','RGB w/o Alpha'])
    dsimg=rp.horizontally_concatenated_images(ds_imgs)
    dsimg=rp.labeled_image(dsimg,X,size=20)
    rp.display_image(dsimg)
    

    # rp.line_graph_via_bokeh([x.var() for x  in ds_black])
    # rp.line_graph_via_bokeh([x.mean() for x  in ds_black])

    icecream.ic(len(ds_black),X)
    if not img_only:
        rp.display_image_slideshow([rp.labeled_image(rp.horizontally_concatenated_images(imgs),i*skip*SKIP) for i,imgs in enumerate(zip(ds_black[::skip],ds_white[::skip],alphas[::skip],fulls[::skip]))])
        
print(X)

In [ ]:
image_black=ColorBlendedLearnableImage(image_url,(0,0,0),learnable_image=None                       ).to(s.device)
image_white=ColorBlendedLearnableImage(image_url,(1,1,1),learnable_image=image_black.learnable_image).to(s.device)
image_red  =ColorBlendedLearnableImage(image_url,(1,0,0),learnable_image=image_black.learnable_image).to(s.device)
image_green=ColorBlendedLearnableImage(image_url,(0,1,0),learnable_image=image_black.learnable_image).to(s.device)
image_blue =ColorBlendedLearnableImage(image_url,(0,0,1),learnable_image=image_black.learnable_image).to(s.device)

def image_alpha():
    return rp.as_numpy_array(image_black.learnable_image()[3])

def image_full():
    return rp.as_numpy_image(image_black.learnable_image()[:3])

params=list(image_black.parameters())
icecream.ic(len(params))
optim=torch.optim.Adam(params,lr=1e-3/5)#For fourier. 1e-2 is too high.
# optim=torch.optim.Adam(params,lr=1e-2)#Normally this is too high, as the colors start to get too strong...but in this case, we want a binary mask anyway lol - might as well do it the fast way!
# optim=torch.optim.Adam(params,lr=1e-2/5)#Normally this is too high, as the colors start to get too strong...but in this case, we want a binary mask anyway lol - might as well do it the fast way!
# optim=torch.optim.Adam(params,lr=1e-1)#For raster. 
# optim=torch.optim.SGD(params,lr=1e-1)#For raster
# optim=torch.optim.SGD(params,lr=1e-5)#For raster

ds_black=[]
ds_white=[]
ds_red  =[]
ds_green=[]
ds_blue =[]
alphas=[]
fulls=[]
alpha_sums=[]
iter_num=0

print(X)

In [ ]:
SKIP=10

ALPHA_HATE=1e-1
ALPHA_HATE=0

et=rp.eta(10000)
try:
    for _ in range(10000):
        et(_)
        iter_num+=1

        speedup=2#1 for slow and 5 for fastest. 2 works well.

        includes=rp.random_batch(range(5),speedup)
        if 0 in includes: s.train_step(text_black,image_black()[None],by_loss=False)
        if 1 in includes: s.train_step(text_white,image_white()[None],by_loss=False)
        if 2 in includes: s.train_step(text_red  ,image_red  ()[None],by_loss=False)
        if 3 in includes: s.train_step(text_green,image_green()[None],by_loss=False)
        if 4 in includes: s.train_step(text_blue ,image_blue ()[None],by_loss=False)

        # (image_black.learnable_image()[3].mean()*100000).backward(retain_graph=True) #Make alpha small
        alpha_sum=image_black.learnable_image()[3].sum()
        alpha_sums.append(float(alpha_sum))
        (alpha_sum*ALPHA_HATE).backward(retain_graph=True) #Make alpha small

        # s.train_step(text_black,image_black.learnable_image()[3:].repeat(3,1,1)[None],by_loss=False) #Run it on the alpha channel too!!



        with torch.no_grad():
            if not _%SKIP:
                ds_black.append(rp.as_numpy_image(image_black()))
                ds_white.append(rp.as_numpy_image(image_white()))
                fulls.append(rp.as_numpy_image(image_full()))
                alphas.append(image_alpha())
                # ds_red  .append(rp.as_numpy_image(image_red  ()))
                # ds_green.append(rp.as_numpy_image(image_green()))
                # ds_blue .append(rp.as_numpy_image(image_blue ()))
            if not _%50:
                do_display(img_only=True)
                icecream.ic(iter_num,float(alpha_sum))
            elif not _%500:
                ...
                from IPython.display import clear_output
                clear_output()
                # import IPython
                # IPython.

        optim.step()
        optim.zero_grad()
except KeyboardInterrupt:
    rp.line_graph_via_bokeh(alpha_sums,ylabel='Total Alpha',xlabel='Iteration')
    

In [ ]:
do_display(skip=10)

In [ ]:
do_display(skip=10)

In [ ]:
do_display(skip=10)

In [ ]:
do_display(10)

In [ ]:
do_display(skip=10)

In [ ]:
do_display(skip=10)

In [ ]:
do_display(skip=10)

In [ ]:
do_display(skip=10)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)

In [ ]:
do_display(skip=5)